In [5]:
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
# Install pytorch using wheel

!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58eec000 @  0x7fc1590712a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 20.8MB/s 
    100% |████████████████████████████████| 2.0MB 4.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [10]:
# Install the github repo
!git clone https://github.com/jaderabbit/UnsupervisedMT.git


Cloning into 'UnsupervisedMT'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 95 (delta 13), reused 19 (delta 8), pack-reused 71
Unpacking objects: 100% (95/95), done.
/bin/bash: ./get_my_data.sh: No such file or directory


In [0]:
# cd into github repo
# Run preparation
!cd UnsupervisedMT/NMT; ./get_my_data.sh zu https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu


===== Input parameters
 source: en
 target: zu
 target monolingual url: https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu
 
Cloning Moses from GitHub repository...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 147198, done.
remote: Total 147198 (delta 0), reused 0 (delta 0), pack-reused 147198
Receiving objects: 100% (147198/147198), 129.67 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (113765/113765), done.
Moses found in: /content/UnsupervisedMT/NMT/tools/mosesdecoder
Cloning fastBPE from GitHub repository...
Cloning into 'fastBPE'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.
fastBPE found in: /content/UnsupervisedMT/NMT/tools/fastBPE
Compiling fastBPE...
fastBPE compiled in: /content/UnsupervisedMT/NMT/tools/fastBPE/fast
Cloning fastText from GitHub repository...
Cloning into 'fastText'...
remote: Enumerating objects: 91, done.
remote

In [16]:
!ls data/mono
!ls data/para

bpe_codes    mono.en.tok.60000	mono.zu.tok.60000	  vocab.en-zu.60000
mono.en      mono.zu		news.2007.en.shuffled.gz  vocab.zu.60000
mono.en.tok  mono.zu.tok	vocab.en.60000


In [0]:
#!/bin/bash
## With
#usage: run_unsupervised_nmt.sh 
!TARGET=zu
!DUMP_PATH=
!MONO_DATASET="en:./data/mono/mono.en.tok.60000.pth,,;$TARGET:./data/mono/mono.$TARGET.tok.60000.pth,,"
!PARA_DATASET="en-$TARGET:,./data/para/en$TARGET_parallel.dev.en.60000.pth,./data/para/en$TARGET_parallel.dev.$TARGET.60000.pth"
!PRETRAINED="./data/mono/all.en-$TARGET.60000.vec"

!echo ""
!echo "===== Input parameters ====="
!echo " TARGET       : $TARGET"
!echo " MONO_DATASET : $MONO_DATASET"
!echo " PARA_DATASET : $PARA_DATASET"
!echo " PRETRAINED   : $PRETRAINED"
!echo " "
!cd UnsupervisedMT/NMT; python main.py --exp_name en$TARGET \
!    --transformer True  --n_enc_layers 4 --n_dec_layers 4  \
!    --share_enc 3  --share_dec 3  --share_lang_emb True  --share_output_emb True \
!    --langs "en,$TARGET" --n_mono -1  --mono_dataset $MONO_DATASET  --para_dataset $PARA_DATASET \
!    --mono_directions "en,$TARGET"  --word_shuffle 3 --word_dropout 0.1 --word_blank 0.2 \
!    --pivo_directions "en-$TARGET-en,$TARGET-en-$TARGET" \
!    --pretrained_emb $PRETRAINED  --pretrained_out True \
!    --lambda_xe_mono '0:1,100000:0.1,300000:0' --lambda_xe_otfd 1 \
!    --otf_num_processes 30   --otf_sync_params_every 1000 \
!    --enc_optimizer adam,lr=0.0001 \
!    --group_by_size True \
!    --batch_size 32 \
!    --epoch_size 500000 --stopping_criterion bleu_en_${TARGET}_valid,10 \
!    --freeze_enc_emb False --freeze_dec_emb False \
!    --dump_path $DUMP_PATH        

